In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
import os
import numpy as np
import wandb
import h5py
import matplotlib.pyplot as plt

In [4]:
# api = wandb.Api()
# wandb.init(project='caueeg-task2', id='gjkysllw', resume='must')
# wandb.run.finish()

Learning Rate,0.0
Loss,0.23025
Multi-Crop Test Accuracy,65.62147
Test Accuracy,61.26412
Train Accuracy,91.87914
Validation Accuracy,54.98599


In [ ]:
api = wandb.Api()
runs = api.runs('ipis-mjkim/caueeg-task1')

for run in runs:
    # print(run, run.config['model'])
    # if run.name == '1tdyketc':
    if run.name == '18lkfaaw':
    # if run.state == "finished":
        print(run.name)

        confusion = np.array(run.summary['Confusion Matrix (Array)'])
        n_classes = confusion.shape[0]

        accuracy = np.zeros((n_classes,))
        sensitivity = np.zeros((n_classes,))
        specificity = np.zeros((n_classes,))
        precision = np.zeros((n_classes,))
        recall = np.zeros((n_classes,))
        f1_score = np.zeros((n_classes,))

        for c in range(n_classes):
            tp = confusion[c, c]
            fn = confusion[c].sum() - tp
            fp = confusion[:, c].sum() - tp
            tn = confusion.sum() - tp - fn - fp

            accuracy[c] = (tp + tn) / (tp + fn + fp + tn)
            sensitivity[c] = tp / (tp + fn)
            specificity[c] = tn / (fp + tn)
            precision[c] = tp / (tp + fp)
            recall[c] = tp / (tp + fn)

        f1_score = 2 * precision * recall / (precision + recall)
        
        print(run.summary['Test Accuracy'])
        print('accuracy:', accuracy)
        print('sensitivity:', sensitivity)
        print('specificity:', specificity)
        print('precision:', precision)
        print('recall:', recall)
        print('f1_score:', f1_score)
        break

18lkfaaw
73.06372549019608
accuracy: [0.73063725 0.73063725]
sensitivity: [0.54764493 0.82416667]
specificity: [0.82416667 0.54764493]
precision: [0.61418123 0.78092648]
recall: [0.54764493 0.82416667]
f1_score: [0.57900785 0.80196414]


In [ ]:
# api = wandb.Api()
# runs = api.runs('ipis-mjkim/eeg-analysis')

# for run in runs:
#     print(run)
#     debug_table_serial = run.summary['Test Debug Table/Serial']
#     debug_table_edf = run.summary['Test Debug Table/EDF']
#     debug_table_pred = run.summary['Test Debug Table/Pred']
#     debug_table_gt = run.summary['Test Debug Table/GT']
    
#     fig = plt.figure(num=1, clear=True, figsize=(20.0, 4.0), constrained_layout=True)
#     ax = fig.add_subplot(1, 1, 1)

#     total_error, total_count = (0, 0)

#     for edf in np.unique(debug_table_edf):
#         indices = [i for i, x in enumerate(debug_table_edf) if x == edf]

#         err, cnt = (0, 0)
#         for i in indices:
#             cnt += sum(debug_table_pred[i])
#             err += sum(debug_table_pred[i]) - debug_table_pred[i][debug_table_gt[i]]

#         total_error += err
#         total_count += cnt

#         ax.bar(edf, err / cnt, color=['g', 'b', 'r'][debug_table_gt[i]])

#     ax.set_title(f'Test Debug Table (Acc. {1.0 - total_error / total_count: .2f}%)', fontsize=18)
#     ax.set_ylim(0.0, 1.0)
#     plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)

#     # run.summary['Test Debug Table (Image)'] = wandb.Image(plt)
#     # run.summary.update()
#     print(type(wandb.Image(plt)))
#     run.summary.update({'Test Debug Table (Image)': wandb.Image(plt)})
#     fig.clear()
#     plt.close(fig)

In [ ]:
# api = wandb.Api()
# run = api.run("ipis-mjkim/eeg-analysis/1nhn425l")
# print(run)

# run.summary.update()

In [ ]:
# api = wandb.Api()
# runs = api.runs('ipis-mjkim/eeg-analysis')

# for run in runs:
#     print(run)
#     if 'Age' in run.config:
#         del run.config['Age']
#         run.config.update()
#         run.update()        
    
#     if 'no-age' not in run.config['model']:
#         run.config['use_age'] = 'fc'
#         run.config.update()
#         run.update()
#     else:
#         run.config['use_age'] = None
#         run.config.update()
#         run.update()        

In [ ]:
# api = wandb.Api()
# runs = api.runs('ipis-mjkim/eeg-analysis')

# for run in runs:
#     print(run)
#     debug_table_serial = run.summary['Test Debug Table/Serial']
#     debug_table_edf = run.summary['Test Debug Table/EDF']
#     debug_table_pred = run.summary['Test Debug Table/Pred']
#     debug_table_gt = run.summary['Test Debug Table/GT']
    
#     fig = plt.figure(num=1, clear=True, figsize=(20.0, 4.0), constrained_layout=True)
#     ax = fig.add_subplot(1, 1, 1)

#     total_error, total_count = (0, 0)

#     for edf in np.unique(debug_table_edf):
#         indices = [i for i, x in enumerate(debug_table_edf) if x == edf]

#         err, cnt = (0, 0)
#         for i in indices:
#             cnt += sum(debug_table_pred[i])
#             err += sum(debug_table_pred[i]) - debug_table_pred[i][debug_table_gt[i]]

#         total_error += err
#         total_count += cnt

#         ax.bar(edf, err / cnt, color=['g', 'b', 'r'][debug_table_gt[i]])

#     ax.set_title(f'Test Debug Table (Acc. {1.0 - total_error / total_count: .2f}%)', fontsize=18)
#     ax.set_ylim(0.0, 1.0)
#     plt.setp(ax.get_xticklabels(), rotation=90, ha="right", fontsize=9, visible=True)

#     # run.summary['Test Debug Table (Image)'] = wandb.Image(plt)
#     # run.summary.update()
#     print(type(wandb.Image(plt)))
#     run.summary.update({'Test Debug Table (Image)': wandb.Image(plt)})
#     fig.clear()
#     plt.close(fig)